In [ ]:
!pip install mediapipe
!pip install tensorflow

In [3]:
from matplotlib import pyplot as plt
import time
import tensorflow as tf
import math

from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential, Model

import os
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import pickle


In [ ]:
# Function to extract landmarks from a video file
def extract_landmarks(video_path):
    # Use Mediapipe to extract landmarks from each frame of the video
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(video_path)
    landmarks = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        if results.pose_landmarks:
            landmark_list = [lm.x for lm in results.pose_landmarks.landmark]  # Example: Use x-coordinates of landmarks
            landmarks.append(landmark_list)
    cap.release()
    return landmarks

# Path to the directory containing the gym exercise videos
data_dir = "Workout_Video"

result_dir = "Workout_Video_Extracts"

# List of gym exercise classes (subfolders in the data directory)
exercise_classes = os.listdir(data_dir)

print("Preparing Dataset")
# Prepare the dataset
for i, exercise_class in enumerate(exercise_classes):
    class_dir = os.path.join(data_dir, exercise_class)

    output_dir = os.path.join(result_dir, exercise_class)
    result_file_path_x = output_dir + "+X.txt"
    result_file_path_y = output_dir + "+y.txt"

    filenamey = exercise_class + "+y.txt"

    if filenamey in os.listdir(result_dir):
      continue
    print(class_dir)

    xin, yin = [], []
    for video_file in os.listdir(class_dir):
        video_path = os.path.join(class_dir, video_file)
        landmarks = extract_landmarks(video_path)
        xin.append(landmarks)
        yin.append(i)

    with open(result_file_path_x, "wb") as filex:
      pickle.dump(xin, filex)
    with open(result_file_path_y, "wb") as filey:
      pickle.dump(yin, filey)


In [ ]:
X, y = [], []
for exercise_class in exercise_classes:
    print(exercise_class)
    output_dir = os.path.join(result_dir, exercise_class)
    result_file_path_x = output_dir + "+X.txt"
    result_file_path_y = output_dir + "+y.txt"

    with open(result_file_path_x, "rb") as filex:
      for inpx in pickle.load(filex):
        X.append(inpx)
    with open(result_file_path_y, "rb") as filey:
      for inpy in pickle.load(filey):
        y.append(inpy)


In [ ]:
print("splitting Dataset")
# Split the dataset into training, validation, and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

num_landmarks = 33

print("Padding or truncating dataset")
# Pad or truncate sequences to ensure they have the same length

fixed_seq_length = 250


X_train = [seq[:len(seq)] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in X_train]
X_val = [seq[:len(seq)] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in X_val]
X_test = [seq[:len(seq)] + [[0] * num_landmarks] * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in X_test]


In [ ]:
# Convert to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [6]:
sequence_length = 30
num_input_values = 132
actions = np.array(['curl', 'press', 'squat'])

In [7]:
lstm = Sequential()
lstm.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(sequence_length, num_input_values)))
lstm.add(LSTM(256, return_sequences=True, activation='relu'))
lstm.add(LSTM(128, return_sequences=False, activation='relu'))
lstm.add(Dense(128, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 128)           133632    
                                                                 
 lstm_4 (LSTM)               (None, 30, 256)           394240    
                                                                 
 lstm_5 (LSTM)               (None, 128)               197120    
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 3)                 195       
                                                                 
Total params: 749955 (2.86 MB)
Trainable params: 74995

In [ ]:
lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lstm.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

In [8]:
model = lstm
model_name = "LSTM"
yhat = model.predict(X_test, verbose=0)

# Get list of classification predictions
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

# Model accuracy
classification_accuracy = accuracy_score(ytrue, yhat)
print(f"{model_name} classification accuracy = {round(classification_accuracy*100,3)}%")

LSTM classification accuracy = 86.667%


In [10]:

yhat = model.predict(X_test, verbose=0)

# Get list of classification predictions
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

# Precision, recall, and f1 score
report = classification_report(ytrue, yhat, target_names=actions, output_dict=True)

precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1_score = report['weighted avg']['f1-score']

print(f"{model_name} weighted average precision = {round(precision,3)}")
print(f"{model_name} weighted average recall = {round(recall,3)}")
print(f"{model_name} weighted average f1-score = {round(f1_score,3)}\n")


LSTM weighted average precision = 0.894
LSTM weighted average recall = 0.867
LSTM weighted average f1-score = 0.868
